In [ ]:
# Import third library
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pyquery import PyQuery as pq
import time
import openpyxl as op              
# Global Variable
count = 1                           
 
# Setting browser options
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ['enable-automation'])
driver = webdriver.Chrome(options=options)
driver.maximize_window()
wait = WebDriverWait(driver,10)
# The web will stop for 10s, login by scaning QR code at this time
 
# Input "Keyword", search and begin to scrape
def search_goods(KEYWORD,start_page,total_pages):
    try:
        print('正在搜索: ')
        driver.get('https://www.taobao.com')
        # time.sleep(10)           
        driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument",
                               {"source": """Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"""})
        # Find input box of "searching"
        input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#q")))
        # Find button of "searching"
        submit = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '#J_TSearchForm > div.search-button > button')))
        # Input "Keyword" to input box
        input.send_keys(KEYWORD)
        # Click "searching" button
        submit.click()
 
        # If "startpage" is not 1, scroll to the bottom of page manually
        if start_page != 1:
            time.sleep(3)
            # Find input box of "page", input "startpage"
            pageInput = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="pageContent"]/div[1]/div[3]/div[4]/div/div/span[3]/input')))
            pageInput.send_keys(start_page)
            # Find and click button of "turning"
            admit = wait.until(EC.element_to_be_clickable(
                (By.XPATH, '//*[@id="pageContent"]/div[1]/div[3]/div[4]/div/div/button[3]')))
            admit.click()
 
        # Get data of products
        get_goods(start_page)
 
        # Page turning
        for i in range(start_page + 1, total_pages+1):
            page_turning(i)
 
    except TimeoutException:
        print("search_goods: error")
        return search_goods(KEYWORD,start_page,total_pages)
 
# Page turning Function
def page_turning(page_number):
    print('正在翻页: ', page_number)
    try:
        time.sleep(2)
        # Find and click the button of "next page"
        submit = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="search-content-leftWrap"]/div[3]/div[4]/div/div/button[2]/span')))
        submit.click()
        # Judge if the page number correct
        wait.until(EC.text_to_be_present_in_element((By.XPATH, '//*[@id="search-content-leftWrap"]/div[3]/div[4]/div/div/span[1]/em'), str(page_number)))
        get_goods(page_number)
    except TimeoutException:
        print("page_number: error")
        page_turning(page_number)
 
# Get products data of each page
def get_goods(page):
    # Claim global varibale
    global count
 
    # Wait until whole page load completely by scrolling to bottom
    if input('确认界面加载完毕，输入数字“1”开始爬取-->') == 1:
        pass
 
    html = driver.page_source
    doc = pq(html)
    # Extract common parent elements of all items
    items = doc('div.contentInner--xICYBlag > a').items()
 
    for item in items:
        # Locate title
        title = item.find('.title--qJ7Xg_90 span').text()
        # Locate price
        price_int = item.find('.priceInt--yqqZMJ5a').text()
        price_float = item.find('.priceFloat--XpixvyQ1').text()
        if price_int and price_float:
            price = float(f"{price_int}{price_float}")
        else:
            price = 0.0
        # Locate deal
        deal = item.find('.realSales--XZJiepmt').text()
        # Locate location
        location = item.find('.procity--wlcT2xH9 span').text()
        # Locate shop
        shop = item.find('.shopNameText--DmtlsDKm').text()
        # Locate if exemption from postage
        postText = item.find('.subIconWrapper--Vl8zAdQn').text()
        postText = "包邮" if "包邮" in postText else "/"
        # Locate product URL
        t_url = item.attr('href')
        # Locate shop URL
        shop_url = item.find('.TextAndPic--grkZAtsC a')
        shop_url = shop_url.attr('href')
        # Locate picture URL
        img = item.find('.mainPicWrapper--qRLTAeii img')
        img_url = img.attr('src')

        product = {
            'Page': page,
            'Num': count-1,
            'title': title,
            'price': price,
            'deal': deal,
            'location': location,
            'shop': shop,
            'isPostFree': postText,
            'url': t_url,
            'shop_url': shop_url,
            'img_url': img_url
        }
        print(product)
 
        # Write data in Excel
        wb.cell(row=count, column=1, value=page)                
        wb.cell(row=count, column=2, value=count-1)            
        wb.cell(row=count, column=3, value=title)               
        wb.cell(row=count, column=4, value=price)               
        wb.cell(row=count, column=5, value=deal)                
        wb.cell(row=count, column=6, value=location)            
        wb.cell(row=count, column=7, value=shop)                
        wb.cell(row=count, column=8, value=postText)           
        wb.cell(row=count, column=9, value=t_url)               
        wb.cell(row=count, column=10, value=shop_url)          
        wb.cell(row=count, column=11, value=img_url)           
        count += 1                                             
 
# Main function
def Crawer_main(KEYWORD,pageStart,pageEnd):
    try:
        search_goods(KEYWORD,pageStart,pageEnd)
    except Exception as exc:
        print('Crawer_main函数报错:', exc)
 
 
 
 
if __name__ == '__main__':
 
    KEYWORD = input('输入搜索的商品关键词Keyword：')           
    pageStart = int(input('输入爬取的起始页PageStart：'))    
    pageEnd = int(input('输入爬取的终止页PageEnd：'))        
 
    # Establish Excel file
    try:
        ws = op.Workbook()                                  
        wb = ws.create_sheet(index=0)                  
        # Excel第一行：表头
        title_list = ['Page', 'Num', 'title', 'Price', 'Deal', 'Location', 'Shop', 'IsPostFree', 'Title_URL',
                      'Shop_URL', 'Img_URL']
        for i in range(0, len(title_list)):
            wb.cell(row=count, column=i + 1, value=title_list[i])
        count += 1  # Write data in next line
    except Exception as exc:
        print("Excel建立失败！")
 
    # Start to scrape
    Crawer_main(KEYWORD,pageStart,pageEnd)
 
    # Save Excel file
    Filename = input("输入存储文件名称：")
    Filename = Filename + '_From_Taobao.xlsx'
    ws.save(filename = Filename)
    print(Filename + "存储成功~")